### Variational Inference
아래 페이지를 요약한다.
- https://ermongroup.github.io/cs228-notes/inference/variational/

> - 어떤 미지의 확률분포 $p$에 근접한 분포 $q$를 추론하는 문제가 있다고 할 때, 이를 풀 수 있는 방법들중 하나가 Variational inference이다. 결론부터 말하자면 분포에 대한 추론(inference) 문제를 최적화(optimization) 문제로 바꿔 해결하는 방법이다. 이런 문제를 MCMC와 같은 샘플링으로도 접근할 수 있으나 샘플링을 이용하는 경우 얻은 솔루션이 얼마나 $p$에 아까운지 알기 어렵고, 적절한 샘플링 기법을 선택하는 것 자체가 어렵기도 하다.
- Variational inference에서는 두 분포의 KL divergence를 이용한다. 참고로 이산 분포 $p$와 $q$의 KL divergence는 아래와 같이 정의된다.
 - 이 값은 $p$와 $q$에 무관하게 항상 0보다 크거나 같고, 만약 두 분포가 같다면 이 값은 0이 된다.
$$KL(q \Vert p) = \sum_x q(x) \log \frac{q(x)}{p(x)}$$

> - Variational inference에서는 미지의 분포 $p$를 아래와 같이 표현할 수 있다고 가정한다.
$$p(x_1, \cdots, x_n; \theta) = \frac{\tilde{p}(x_1, \cdots, x_n; \theta)}{Z(\theta)} = \frac{1}{Z(\theta)} \prod_k \phi_k(x_k;\theta)$$
- $KL(q \Vert p)$를 최소화 하는 $q$를 찾는대신 위 표현을 이용해 $KL(q \Vert \tilde{p}) = J(q)$를 최소화하는 $q$를 찾는다. 참고로 $KL(q \Vert p)$와 $KL(q \Vert \tilde{p})$는 아래와 같은 관계가 있다.
$$\log Z(\theta) = KL(q \Vert p) - J(q)$$
- 그런데 $KL(q \Vert p)$은 0보다 크거나 같으므로 $- J(q)$은 $\log Z(\theta)$의 lower bound 이다.($Z(\theta)$를 최대화하면 evidence의 log-likelihood가 최대화 된다.)
$$\begin{align}
\log Z(\theta) &= KL(q \Vert p) - J(q) \\
&\geq -J(q) \\
&= \sum_x q(x) \log \frac{q(x)}{\tilde{p}(x)} \\
&= \mathbb{E}_{q(x)}\bigg[ \log\tilde{p}(x) - \log q(x) \bigg]
\end{align}$$

> - 그렇다면 어떤 형태의 $q$함수를 사용해야할까? 지수족 함수, 신경망, 가우시안 과정, 잠재변수 모형 등 다양한 선택지가 있다. 이들 중 아래와 같은 형태의 $q(x)$를 사용하는 방법을 'Mean-field inference'라 한다. (여기서 $q_i(x_i)$는 이산형 확률변수 $x_i$에 대한 확률분포이다.)
$$q(x) = q_1(x_1) q_2(x_2) \cdots q_n(x_n)$$
- Mean-field inferece는 아래 목적함수를 최소화 하는 식으로 풀게 된다. (이를 푸는 방법으로 $q_j$를 하나식 선택하고 나머지 $q_{\cdot}$는 고정한체로 $J(q)$를 최소화하는 $q_j$를 찾을 수 있는데, 이런 방법을 coordinate descent라 한다.)
$$\min_{q_1, \cdots, q_j, \cdots, q_n}J(q)$$

###### Variational Auto-Encoder
아래 페이지의 'Auto-encoding variational Bayes'부터...
- https://ermongroup.github.io/cs228-notes/extras/vae/